In [1]:
%run spark_utilities.ipynb

following variables have been created:
db_name
jdbcURI
driver


In [2]:
%run custom_methods.ipynb

custom methods:
normalize_column_name
download_files


# *Import the datasets*

In [3]:
file_population = "datasets/population_by_age.tsv"
country_lookup = "datasets/country_lookup.csv"

if os.path.isfile(file_population):
    df_population_sc = spark.read.csv(file_population, header=True, inferSchema=True, sep=r'\t')
else:
    print("File {0} not found".format(file_population))


if os.path.isfile(country_lookup):
    df_country_lookup_sc = spark.read.csv(country_lookup, header=True, inferSchema=True)
else:
    print("File {0} not found".format(country_lookup))

In [4]:
df_population_sc.show()

+------------------+------+-----+------+------+------+-----+------+------+-----+------+------+------+
| indic_de,geo\time| 2008 |2009 | 2010 | 2011 | 2012 |2013 | 2014 | 2015 |2016 | 2017 | 2018 | 2019 |
+------------------+------+-----+------+------+------+-----+------+------+-----+------+------+------+
|       PC_Y0_14,AD| 14.6 |14.5 | 14.5 | 15.5 | 15.5 |15.5 |    : |    : |   : |    : |    : | 13.9 |
|       PC_Y0_14,AL| 24.1 |23.3 | 22.5 | 21.6 | 20.7 |20.1 | 19.6 | 19.0 |18.5 | 18.2 | 17.7 | 17.2 |
|       PC_Y0_14,AM| 19.0 |18.6 | 18.3 |    : |    : |   : |    : | 19.4 |19.6 | 20.0 | 20.2 | 20.2 |
|       PC_Y0_14,AT| 15.4 |15.1 | 14.9 | 14.7 | 14.6 |14.4 | 14.3 | 14.3 |14.3 | 14.4 | 14.4 | 14.4 |
|       PC_Y0_14,AZ| 23.2 |22.6 | 22.6 | 22.3 | 22.2 |22.3 | 22.4 | 22.4 |22.5 | 22.6 | 22.6 | 22.4 |
|       PC_Y0_14,BE| 16.9 |16.9 | 16.9 |17.0 b| 17.0 |17.0 | 17.0 | 17.0 |17.0 | 17.0 | 17.0 | 16.9 |
|       PC_Y0_14,BG| 13.1 |13.1 | 13.2 | 13.2 | 13.4 |13.6 | 13.7 | 13.9 |14.0 | 1

In [16]:
df_country_lookup_sc.show()

+--------------------+--------------------+--------------------+---------+----------+
|             country|country_code_2_digit|country_code_3_digit|continent|population|
+--------------------+--------------------+--------------------+---------+----------+
|               Aruba|                  AW|                 ABW|  America|    106766|
|         Afghanistan|                  AF|                 AFG|     Asia|  38928341|
|              Angola|                  AO|                 AGO|   Africa|  32866268|
|            Anguilla|                  AI|                 AIA|  America|     15002|
|             Albania|                  AL|                 ALB|   Europe|   2862427|
|             Andorra|                  AD|                 AND|   Europe|     76177|
|United Arab Emirates|                  AE|                 ARE|     Asia|   9890400|
|           Argentina|                  AR|                 ARG|  America|  45195777|
|             Armenia|                  AM|           

# *Normalize cols for df_population_sc*

In [5]:
# the columns are not well formatted! use the normalize pattern in custom_methods!
df_population_sc.columns

['indic_de,geo\\time',
 '2008 ',
 '2009 ',
 '2010 ',
 '2011 ',
 '2012 ',
 '2013 ',
 '2014 ',
 '2015 ',
 '2016 ',
 '2017 ',
 '2018 ',
 '2019 ']

In [6]:
# also the types for the years are strings and should be decimal(4,2)
df_population_sc.dtypes

[('indic_de,geo\\time', 'string'),
 ('2008 ', 'string'),
 ('2009 ', 'string'),
 ('2010 ', 'string'),
 ('2011 ', 'string'),
 ('2012 ', 'string'),
 ('2013 ', 'string'),
 ('2014 ', 'string'),
 ('2015 ', 'string'),
 ('2016 ', 'string'),
 ('2017 ', 'string'),
 ('2018 ', 'string'),
 ('2019 ', 'string')]

In [7]:
df_population_norm = normalize_column_name(df_population_sc)
df_population_norm.show()

+------------------+------+-----+------+------+------+-----+------+------+-----+------+------+------+
| indic_de,geo_time|  2008| 2009|  2010|  2011|  2012| 2013|  2014|  2015| 2016|  2017|  2018|  2019|
+------------------+------+-----+------+------+------+-----+------+------+-----+------+------+------+
|       PC_Y0_14,AD| 14.6 |14.5 | 14.5 | 15.5 | 15.5 |15.5 |    : |    : |   : |    : |    : | 13.9 |
|       PC_Y0_14,AL| 24.1 |23.3 | 22.5 | 21.6 | 20.7 |20.1 | 19.6 | 19.0 |18.5 | 18.2 | 17.7 | 17.2 |
|       PC_Y0_14,AM| 19.0 |18.6 | 18.3 |    : |    : |   : |    : | 19.4 |19.6 | 20.0 | 20.2 | 20.2 |
|       PC_Y0_14,AT| 15.4 |15.1 | 14.9 | 14.7 | 14.6 |14.4 | 14.3 | 14.3 |14.3 | 14.4 | 14.4 | 14.4 |
|       PC_Y0_14,AZ| 23.2 |22.6 | 22.6 | 22.3 | 22.2 |22.3 | 22.4 | 22.4 |22.5 | 22.6 | 22.6 | 22.4 |
|       PC_Y0_14,BE| 16.9 |16.9 | 16.9 |17.0 b| 17.0 |17.0 | 17.0 | 17.0 |17.0 | 17.0 | 17.0 | 16.9 |
|       PC_Y0_14,BG| 13.1 |13.1 | 13.2 | 13.2 | 13.4 |13.6 | 13.7 | 13.9 |14.0 | 1

In [8]:
df_country_lookup_sc.show()

+--------------------+--------------------+--------------------+---------+----------+
|             country|country_code_2_digit|country_code_3_digit|continent|population|
+--------------------+--------------------+--------------------+---------+----------+
|               Aruba|                  AW|                 ABW|  America|    106766|
|         Afghanistan|                  AF|                 AFG|     Asia|  38928341|
|              Angola|                  AO|                 AGO|   Africa|  32866268|
|            Anguilla|                  AI|                 AIA|  America|     15002|
|             Albania|                  AL|                 ALB|   Europe|   2862427|
|             Andorra|                  AD|                 AND|   Europe|     76177|
|United Arab Emirates|                  AE|                 ARE|     Asia|   9890400|
|           Argentina|                  AR|                 ARG|  America|  45195777|
|             Armenia|                  AM|           

In [9]:
df_country_lookup_sc.columns

['country',
 'country_code_2_digit',
 'country_code_3_digit',
 'continent',
 'population']

In [11]:
df_country_lookup_sc.dtypes

[('country', 'string'),
 ('country_code_2_digit', 'string'),
 ('country_code_3_digit', 'string'),
 ('continent', 'string'),
 ('population', 'int')]

# *Split cols and derive age_group and country_code*

In [12]:
# 1. split the first col => age_group, country_code
# 2. get only 2029 and cast the col since it is a string type
# 3. filter country_code

fields = [
    'age_group'
    ,'country_code'
    ,'percentage_2019'
]

df_population_derived = (
    df_population_norm
    .withColumn("age_group", split(col('indic_de,geo_time'), ',').getItem(0))
    .withColumn("country_code", split(col('indic_de,geo_time'), ',').getItem(1))
    .withColumn("percentage_2019", col('2019').cast(DecimalType(4,2))) # this was a string
    .where(length(col("country_code")) == 2) # remove country_code like EU27_2020 or EA19
    .select(fields)
)

In [13]:
df_population_derived.columns

['age_group', 'country_code', 'percentage_2019']

In [14]:
df_population_derived.dtypes

[('age_group', 'string'),
 ('country_code', 'string'),
 ('percentage_2019', 'decimal(4,2)')]

In [88]:
df_population_derived.orderBy("country_code", "age_group").show()

+----------+------------+---------------+
| age_group|country_code|percentage_2019|
+----------+------------+---------------+
|  PC_Y0_14|          AD|          13.90|
| PC_Y15_24|          AD|          10.60|
| PC_Y25_49|          AD|          39.40|
| PC_Y50_64|          AD|          22.50|
| PC_Y65_79|          AD|          10.20|
|PC_Y80_MAX|          AD|           3.40|
|  PC_Y0_14|          AL|          17.20|
| PC_Y15_24|          AL|          15.50|
| PC_Y25_49|          AL|          33.00|
| PC_Y50_64|          AL|          20.20|
| PC_Y65_79|          AL|          11.40|
|PC_Y80_MAX|          AL|           2.70|
|  PC_Y0_14|          AM|          20.20|
| PC_Y15_24|          AM|          11.80|
| PC_Y25_49|          AM|          36.90|
| PC_Y50_64|          AM|          19.10|
| PC_Y65_79|          AM|           9.00|
|PC_Y80_MAX|          AM|           3.00|
|  PC_Y0_14|          AT|          14.40|
| PC_Y15_24|          AT|          10.90|
+----------+------------+---------

# *Null check*

In [85]:
# check for nulls
df_population_derived.filter(df_population_derived.country_code.isNull()).count()

0

In [84]:
df_population_derived.filter(df_population_derived.percentage_2019.isNull()).count()

36

# *Pivoting of df_population_derived*

In [15]:
# 3. pivoting data => country, country_code_2_digits, contry_code_3_digits, population all the age groups (age_group_0_14 to max)

df_population_pivot = (
    df_population_derived
    .groupby("country_code")
    .pivot("age_group")
    .sum("percentage_2019")
    .orderBy("country_code")
)

In [94]:
df_population_pivot.show()

+------------+--------+---------+---------+---------+---------+----------+
|country_code|PC_Y0_14|PC_Y15_24|PC_Y25_49|PC_Y50_64|PC_Y65_79|PC_Y80_MAX|
+------------+--------+---------+---------+---------+---------+----------+
|          AD|   13.90|    10.60|    39.40|    22.50|    10.20|      3.40|
|          AL|   17.20|    15.50|    33.00|    20.20|    11.40|      2.70|
|          AM|   20.20|    11.80|    36.90|    19.10|     9.00|      3.00|
|          AT|   14.40|    10.90|    34.00|    21.70|    13.80|      5.00|
|          AZ|   22.40|    14.10|    39.10|    17.60|     5.30|      1.50|
|          BE|   16.90|    11.40|    32.70|    20.10|    13.30|      5.60|
|          BG|   14.40|     8.90|    35.00|    20.40|    16.50|      4.80|
|          BY|   16.90|     9.90|    36.60|    21.30|    11.30|      3.90|
|          CH|   15.00|    10.60|    35.00|    20.90|    13.30|      5.20|
|          CY|   16.10|    12.80|    37.10|    17.90|    12.50|      3.70|
|          CZ|   15.90|  

# *Join population and country lookup*

In [18]:
condition = [df_population_pivot.country_code == df_country_lookup_sc.country_code_2_digit]

fields = [
    "country_code_2_digit"
    ,"country_code_3_digit"
    ,"population"
    ,"age_group_0_14"
    ,"age_group_15_24"
    ,"age_group_25_49"
    ,"age_group_50_64"
    ,"age_group_65_79"
    ,"age_group_80_MAX"
]

df_population = (
    df_population_pivot
    .join(df_country_lookup_sc ,how='left', on=condition)
    .withColumnRenamed("PC_Y0_14", "age_group_0_14")
    .withColumnRenamed("PC_Y15_24", "age_group_15_24")
    .withColumnRenamed("PC_Y25_49", "age_group_25_49")
    .withColumnRenamed("PC_Y50_64", "age_group_50_64")
    .withColumnRenamed("PC_Y65_79", "age_group_65_79")
    .withColumnRenamed("PC_Y80_MAX", "age_group_80_MAX")
    .select(fields)
)

In [19]:
df_population.show()

+--------------------+--------------------+----------+--------------+---------------+---------------+---------------+---------------+----------------+
|country_code_2_digit|country_code_3_digit|population|age_group_0_14|age_group_15_24|age_group_25_49|age_group_50_64|age_group_65_79|age_group_80_MAX|
+--------------------+--------------------+----------+--------------+---------------+---------------+---------------+---------------+----------------+
|                  LT|                 LTU|   2794184|         15.10|          10.50|          32.70|          21.90|          14.00|            5.80|
|                  FI|                 FIN|   5517919|         16.00|          11.20|          31.20|          19.90|          16.30|            5.50|
|                  AZ|                 AZE|  10139175|         22.40|          14.10|          39.10|          17.60|           5.30|            1.50|
|                  UA|                 UKR|  43733759|         15.40|           9.60|         

# *Sink table on db*

In [20]:
(
    df_population
    .write
    .mode("overwrite")
    .format("jdbc")
    .options(
        driver=driver
        ,user=usr
        ,password=pwd
        ,url=jdbcURI
        ,dbtable="dbo.Population"
    )
    .save()
)